<a href="https://colab.research.google.com/github/KlymentievMaksym/IntelOb/blob/main/Lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
layers = tf.keras.layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
# from DE import DE

In [30]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [3]:
filepath_save = "./Models/Model_valAcc{val_accuracy:.2f}_valLoss{val_loss:.2f}.keras"
filepath_backup = "./Backups/"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath_save, monitor='val_accuracy', save_best_only=True),
    tf.keras.callbacks.BackupAndRestore(filepath_backup),
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
]

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.RandomContrast(0.2),
])

In [40]:
def DE(pop_size, iterations, function, limits):
    dim = len(limits)
    limits = np.array(limits)
    x_low = limits[:, 0]
    x_high = limits[:, 1]

    population = np.random.uniform(x_low, x_high, (pop_size, dim))

    max_f = -float('inf')
    best_f = float('inf')
    best_pop = np.zeros(dim)

    fitness = np.array([function(X) for X in population])
    for iteration in tqdm(
        range(iterations),
        desc="Processing",
        unit="step",
        bar_format="{l_bar}{bar:40}{r_bar}",
        colour='cyan',
        total=iterations
    ):
        for i in range(pop_size):
            F = np.random.uniform(1e-6, 2)
            P = np.random.uniform(1e-6, 1)
            r = np.random.uniform(1e-6, 1, dim)
            x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            while np.all(population[x1] == population[i]) or np.all(population[x2] == population[i]) or np.all(population[x3] == population[i]):
                x1, x2, x3 = np.random.choice(population.shape[0], size=3, replace=False)
            mutant_vector = population[x1] + F * (population[x2] - population[x3])
            mutant_vector[r < P] = population[i][r < P]
            mutant_fitness = function(mutant_vector)
            if fitness[i] > mutant_fitness:
                fitness[i] = mutant_fitness
                population[i] = mutant_vector.copy()
        el_min = np.argmin(fitness)
        if best_f > fitness[el_min]:
            best_f = fitness[el_min]
            best_pop = population[el_min].copy()
        el_max = np.max(fitness)
        if max_f < el_max:
            max_f = el_max

    return best_f, best_pop

def func(X):
    print(X)
    amount_of_layer_1 = int(round(X[0]))
    amount_of_layer_2 = int(round(X[1]))
    amount_of_layer_3 = int(round(X[2]))
    amount_of_one_dence = int(round(X[3]))
    amount_of_one_filters = int(round(X[4]))
    print(np.int64(np.round(X)))
    print(amount_of_layer_1, amount_of_layer_2, amount_of_layer_3, amount_of_one_dence, amount_of_one_filters)
    model_layers = [
        layers.Input(shape=(32, 32, 3)),
        # data_augmentation
    ]

    for _ in range(amount_of_layer_1):
        model_layers.append(layers.Conv2D(amount_of_one_filters, (3, 3), activation='relu', padding='same'))
        model_layers.append(layers.BatchNormalization())
    model_layers.append(layers.MaxPooling2D(2, 2))

    # for _ in range(amount_of_layer_2):
    #     model_layers.append(layers.Conv2D(amount_of_one_filters * 2, (3, 3), activation='relu', padding='same'))
    #     model_layers.append(layers.BatchNormalization())
    # model_layers.append(layers.MaxPooling2D(2, 2))
    # model_layers.append(layers.Dropout(0.3))

    # for _ in range(amount_of_layer_3):
    #     model_layers.append(layers.Conv2D(amount_of_one_filters * 4, (3, 3), activation='relu', padding='same'))
    #     model_layers.append(layers.BatchNormalization())
    # model_layers.append(layers.MaxPooling2D(4, 4))
    # model_layers.append(layers.Dropout(0.4))

    # model_layers.append(layers.Flatten())
    # for _ in range(amount_of_one_dence):
    #     model_layers.append(layers.Dense(amount_of_one_filters * 8, activation='relu'))
    # model_layers.append(layers.Dropout(0.5))
    model_layers.append(layers.Dense(10, activation='softmax'))

    model = tf.keras.models.Sequential(model_layers)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])
    history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)
    index = np.argmax(history.history['val_accuracy'])
    return (1.0 - history.history['val_accuracy'][index]) + history.history['val_loss'][index]

func_limits = [[0, 4], [0, 4], [0, 4], [0, 4], [4, 128]]


In [41]:
best_f, best_d = DE(2, 4, func, func_limits)

[  2.42565512   2.77224813   0.37218439   2.7574609  125.55249975]
[  2   3   0   3 126]
2 3 0 3 126


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_31 (Conv2D)              │ (None, 32, 32, 126)    │         3,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 32, 32, 126)    │           504 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 32, 32, 126)    │       143,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 32, 32, 126)    │           504 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 16, 16, 126)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 16, 16, 10)     │         1,270 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 148,816 (581.31 KB)

 Trainable params: 148,312 (579.34 KB)

 Non-trainable params: 504 (1.97 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 5 objects could not be loaded. Example error message for object <Conv2D name=conv2d_31, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(3, 3, 3, 126), Received: value.shape=(3, 3, 3, 247). Target variable: <Variable path=sequential_15/conv2d_31/kernel, shape=(3, 3, 3, 126), dtype=float32, value=[[[[-4.80524115e-02 -2.93411054e-02 -3.35196406e-03 ...  1.84877217e-03
     4.70758528e-02  4.60872427e-02]
   [ 2.17325985e-03  2.47298479e-02 -6.91580474e-02 ... -4.57218960e-02
    -4.60505970e-02  6.40663281e-02]
   [ 3.68619412e-02 -6.89198896e-02 -7.05099106e-02 ... -4.31898013e-02
    -6.04745448e-02  3.23898196e-02]]

  [[-6.12484477e-02 -6.46948367e-02  3.59126180e-02 ... -5.14019430e-02
     3.26123759e-02  5.03309816e-03]
   [-1.07288361e-05  2.90596560e-02  2.47314200e-02 ... -2.06970423e-03
    -6.62957877e-03 -3.32197361e-02]
   [-2.04419717e-02 -1.11835450e-02  5.388399

In [ ]:
# model = tf.keras.models.Sequential(
#     [
#         layers.Input(shape=(32, 32, 3)),

#         data_augmentation,

#         layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),

#         layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(2, 2),
#         layers.Dropout(0.3),

#         layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D(4, 4),
#         layers.Dropout(0.4),

#         layers.Flatten(),
#         layers.Dense(1024, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(y_test.shape[1], activation='softmax')
#     ]
# )

# model.summary()
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="Top2")])

In [ ]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks)